In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 00:42:17.978164: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_2_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
7/7 [==============================] - 0s 19ms/step - loss: 15.2019 - mean_squared_error: 15.2019 - rmse: 3.8990 - mean_absolute_error: 2.9997 - val_loss: 11.3276 - val_mean_squared_error: 11.3276 - val_rmse: 3.3657 - val_mean_absolute_error: 2.5786 - lr: 0.0010
Epoch 22/1000
5/7 [====================>.........] - ETA: 0s - loss: 15.9837 - mean_squared_error: 15.9837 - rmse: 3.9980 - mean_absolute_error: 3.1454
Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
7/7 [==============================] - 0s 18ms/step - loss: 16.3270 - mean_squared_error: 16.3270 - rmse: 4.0407 - mean_absolute_error: 3.1721 - val_loss: 12.9039 - val_mean_squared_error: 12.9039 - val_rmse: 3.5922 - val_mean_absolute_error: 2.6676 - lr: 0.0010
Epoch 23/1000
7/7 [==============================] - 0s 17ms/step - loss: 14.9416 - mean_squared_error: 14.9416 - rmse: 3.8654 - mean_absolute_error: 3.0379 - val_loss: 11.4088 - val_mean_squared_error: 11.4088 - val_rmse: 3.377

7/7 [==============================] - 0s 18ms/step - loss: 14.8252 - mean_squared_error: 14.8252 - rmse: 3.8503 - mean_absolute_error: 2.9753 - val_loss: 10.8798 - val_mean_squared_error: 10.8798 - val_rmse: 3.2985 - val_mean_absolute_error: 2.5239 - lr: 5.0000e-04
Epoch 50/1000
7/7 [==============================] - 0s 21ms/step - loss: 13.5481 - mean_squared_error: 13.5481 - rmse: 3.6808 - mean_absolute_error: 2.9113 - val_loss: 10.4069 - val_mean_squared_error: 10.4069 - val_rmse: 3.2260 - val_mean_absolute_error: 2.5065 - lr: 5.0000e-04
Epoch 51/1000
7/7 [==============================] - 0s 19ms/step - loss: 13.5178 - mean_squared_error: 13.5178 - rmse: 3.6766 - mean_absolute_error: 2.8914 - val_loss: 11.0824 - val_mean_squared_error: 11.0824 - val_rmse: 3.3290 - val_mean_absolute_error: 2.5287 - lr: 5.0000e-04
Epoch 52/1000
7/7 [==============================] - 0s 21ms/step - loss: 13.5403 - mean_squared_error: 13.5403 - rmse: 3.6797 - mean_absolute_error: 2.8879 - val_loss: 10

7/7 [==============================] - 0s 19ms/step - loss: 11.0557 - mean_squared_error: 11.0557 - rmse: 3.3250 - mean_absolute_error: 2.5710 - val_loss: 8.7385 - val_mean_squared_error: 8.7385 - val_rmse: 2.9561 - val_mean_absolute_error: 2.2637 - lr: 5.0000e-04
Epoch 79/1000
7/7 [==============================] - 0s 20ms/step - loss: 11.9465 - mean_squared_error: 11.9465 - rmse: 3.4564 - mean_absolute_error: 2.5998 - val_loss: 8.4145 - val_mean_squared_error: 8.4145 - val_rmse: 2.9008 - val_mean_absolute_error: 2.2544 - lr: 5.0000e-04
Epoch 80/1000
7/7 [==============================] - 0s 17ms/step - loss: 11.9460 - mean_squared_error: 11.9460 - rmse: 3.4563 - mean_absolute_error: 2.6681 - val_loss: 8.7858 - val_mean_squared_error: 8.7858 - val_rmse: 2.9641 - val_mean_absolute_error: 2.2588 - lr: 5.0000e-04
Epoch 81/1000
7/7 [==============================] - 0s 20ms/step - loss: 11.2194 - mean_squared_error: 11.2194 - rmse: 3.3495 - mean_absolute_error: 2.6276 - val_loss: 8.2318 -

7/7 [==============================] - 0s 19ms/step - loss: 10.3332 - mean_squared_error: 10.3332 - rmse: 3.2145 - mean_absolute_error: 2.5149 - val_loss: 8.2420 - val_mean_squared_error: 8.2420 - val_rmse: 2.8709 - val_mean_absolute_error: 2.1578 - lr: 5.0000e-04
Epoch 108/1000
7/7 [==============================] - 0s 22ms/step - loss: 10.2204 - mean_squared_error: 10.2204 - rmse: 3.1969 - mean_absolute_error: 2.4995 - val_loss: 7.0369 - val_mean_squared_error: 7.0369 - val_rmse: 2.6527 - val_mean_absolute_error: 2.1401 - lr: 5.0000e-04
Epoch 109/1000
7/7 [==============================] - 0s 18ms/step - loss: 9.4020 - mean_squared_error: 9.4020 - rmse: 3.0663 - mean_absolute_error: 2.3832 - val_loss: 7.9194 - val_mean_squared_error: 7.9194 - val_rmse: 2.8141 - val_mean_absolute_error: 2.1191 - lr: 5.0000e-04
Epoch 110/1000
7/7 [==============================] - 0s 17ms/step - loss: 9.8382 - mean_squared_error: 9.8382 - rmse: 3.1366 - mean_absolute_error: 2.3987 - val_loss: 7.7824 - 

...

Epoch 150/1000
7/7 [==============================] - 0s 17ms/step - loss: 9.2073 - mean_squared_error: 9.2073 - rmse: 3.0344 - mean_absolute_error: 2.3370 - val_loss: 6.8610 - val_mean_squared_error: 6.8610 - val_rmse: 2.6194 - val_mean_absolute_error: 1.9987 - lr: 5.0000e-04
Epoch 151/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.8051 - mean_squared_error: 9.8051 - rmse: 3.1313 - mean_absolute_error: 2.4656 - val_loss: 6.6396 - val_mean_squared_error: 6.6396 - val_rmse: 2.5767 - val_mean_absolute_error: 2.0228 - lr: 5.0000e-04
Epoch 152/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.7475 - mean_squared_error: 9.7475 - rmse: 3.1221 - mean_absolute_error: 2.4137 - val_loss: 6.7928 - val_mean_squared_error: 6.7928 - val_rmse: 2.6063 - val_mean_absolute_error: 1.9932 - lr: 5.0000e-04
Epoch 153/1000
7/7 [==============================] - 0s 16ms/step - loss: 10.1796 - mean_squared_error: 10.1796 - rmse: 3.1906 - mean_absolute_error: 2.4529 - val_lo

7/7 [==============================] - 0s 16ms/step - loss: 9.5508 - mean_squared_error: 9.5508 - rmse: 3.0904 - mean_absolute_error: 2.3730 - val_loss: 6.7429 - val_mean_squared_error: 6.7429 - val_rmse: 2.5967 - val_mean_absolute_error: 2.0069 - lr: 5.0000e-04
Epoch 180/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.0245 - mean_squared_error: 9.0245 - rmse: 3.0041 - mean_absolute_error: 2.3454 - val_loss: 6.6747 - val_mean_squared_error: 6.6747 - val_rmse: 2.5835 - val_mean_absolute_error: 2.0462 - lr: 5.0000e-04
Epoch 181/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.9761 - mean_squared_error: 9.9761 - rmse: 3.1585 - mean_absolute_error: 2.4459 - val_loss: 6.7310 - val_mean_squared_error: 6.7310 - val_rmse: 2.5944 - val_mean_absolute_error: 1.9933 - lr: 5.0000e-04
Epoch 182/1000
7/7 [==============================] - 0s 17ms/step - loss: 9.2023 - mean_squared_error: 9.2023 - rmse: 3.0335 - mean_absolute_error: 2.3426 - val_loss: 6.9186 - val_

Epoch 208/1000
7/7 [==============================] - 0s 17ms/step - loss: 8.8870 - mean_squared_error: 8.8870 - rmse: 2.9811 - mean_absolute_error: 2.2900 - val_loss: 6.6043 - val_mean_squared_error: 6.6043 - val_rmse: 2.5699 - val_mean_absolute_error: 2.0008 - lr: 2.5000e-04
Epoch 209/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.1600 - mean_squared_error: 9.1600 - rmse: 3.0266 - mean_absolute_error: 2.3284 - val_loss: 6.6178 - val_mean_squared_error: 6.6178 - val_rmse: 2.5725 - val_mean_absolute_error: 1.9966 - lr: 2.5000e-04
Epoch 210/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.9821 - mean_squared_error: 8.9821 - rmse: 2.9970 - mean_absolute_error: 2.2765 - val_loss: 6.6057 - val_mean_squared_error: 6.6057 - val_rmse: 2.5702 - val_mean_absolute_error: 2.0093 - lr: 2.5000e-04
Epoch 211/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.4966 - mean_squared_error: 8.4966 - rmse: 2.9149 - mean_absolute_error: 2.2516 - val_loss

Epoch 236/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.7092 - mean_squared_error: 8.7092 - rmse: 2.9511 - mean_absolute_error: 2.2646 - val_loss: 6.5484 - val_mean_squared_error: 6.5484 - val_rmse: 2.5590 - val_mean_absolute_error: 1.9862 - lr: 6.2500e-05
Epoch 237/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.7968 - mean_squared_error: 8.7968 - rmse: 2.9659 - mean_absolute_error: 2.2802 - val_loss: 6.5632 - val_mean_squared_error: 6.5632 - val_rmse: 2.5619 - val_mean_absolute_error: 1.9903 - lr: 6.2500e-05
Epoch 238/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.8420 - mean_squared_error: 8.8420 - rmse: 2.9735 - mean_absolute_error: 2.2550 - val_loss: 6.6048 - val_mean_squared_error: 6.6048 - val_rmse: 2.5700 - val_mean_absolute_error: 2.0022 - lr: 6.2500e-05
Epoch 239/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.9965 - mean_squared_error: 8.9965 - rmse: 2.9994 - mean_absolute_error: 2.3240 - val_loss

Epoch 265/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.4618 - mean_squared_error: 8.4618 - rmse: 2.9089 - mean_absolute_error: 2.2102 - val_loss: 6.6159 - val_mean_squared_error: 6.6159 - val_rmse: 2.5721 - val_mean_absolute_error: 2.0011 - lr: 3.1250e-05
Epoch 266/1000
7/7 [==============================] - 0s 16ms/step - loss: 9.3680 - mean_squared_error: 9.3680 - rmse: 3.0607 - mean_absolute_error: 2.3206 - val_loss: 6.6234 - val_mean_squared_error: 6.6234 - val_rmse: 2.5736 - val_mean_absolute_error: 2.0036 - lr: 3.1250e-05
Epoch 267/1000
7/7 [==============================] - 0s 16ms/step - loss: 7.9205 - mean_squared_error: 7.9205 - rmse: 2.8143 - mean_absolute_error: 2.0919 - val_loss: 6.6263 - val_mean_squared_error: 6.6263 - val_rmse: 2.5742 - val_mean_absolute_error: 2.0075 - lr: 3.1250e-05
Epoch 268/1000
7/7 [==============================] - 0s 16ms/step - loss: 8.2376 - mean_squared_error: 8.2376 - rmse: 2.8701 - mean_absolute_error: 2.2348 - val_loss

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)